In [1]:
using ADCME
using Plots

In [109]:
function f_hi(a, T, R)
    cpHS = Variable(zeros(N, 3))
    for i = 1:N
        scatter_update(cpHS, i, 1, (a[i,2] + a[i,3] * T + a[i,4] * T^2 + a[i,5] * T^3 + a[i,6] * T^4) * R)
        scatter_update(cpHS, i, 2, (a[i,2] + a[i,3] * T / 2 + a[i,4] * T^2 /3 + a[i,5] * T^3 /4 + a[i,6] * T^4 /5 + a[i,7]/T) * R * T)
        scatter_update(cpHS, i, 3, (a[i,2] * log(T) + a[i,3] * T + a[i,4] * T^2 /2 + a[i,5] * T^3 /3 + a[i,6] * T^4 / 4 + a[i,8]) * R)
    end 
    return cpHS
end
 
function f_lo(a, T, R)
    cpHS = Variable(zeros(N, 3))
    for i = 1:N
        scatter_update(cpHS, i, 1, (a[i,9] + a[i,10] * T + a[i,11] * T^2 + a[i,12] * T^3 + a[i,13] * T^4) * R)
        scatter_update(cpHS, i, 2, (a[i,9] + a[i,10] * T / 2 + a[i,11] * T^2 /3 + a[i,12] * T^3 /4 + a[i,13] * T^4 /5 + a[i,14]/T) * R * T)
        scatter_update(cpHS, i, 3,  (a[i,9] * log(T) + a[i,10] * T + a[i,11] * T^2 /2 + a[i,12] * T^3 /3 + a[i,13] * T^4 / 4 + a[i,15]) * R)
    end 
    return cpHS
end 

function compute_falloff(T, pr, a)
    fcent = (1 - a[1]) * exp(-T/a[2]) + a[1] * exp(-T/a[3]) + exp(-a[4]/T)
    c = -0.4 - 0.67 * log(fcent) / log(10)
    n = 0.75 - 1.27 * log(fcent) / log(10)
    f1 = (log(pr) / log(10) + c) / (n - 0.14 *(log(pr) / log(10) + c))
    return  10 ^ ((log(fcent) / log(10)) / (1 + f1 ^ 2))
end

compute_falloff (generic function with 1 method)

In [3]:
using PyCall
#### Read data
py"""
import numpy as np
"""

In [181]:
V = 1  # Volume
m = 0.3778511846534872 # Total mass
tbd = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/tbd.npy") .+ 1
falofr = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/falofr.npy") .+ 1
elmr = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/elmr.npy") .+ 1
NASA_coeffs = py"np.load"("data/NASA_coeffs.npy")
W = py"np.load"("data/molecular_weights.npy") # Molar weight
ν1 = py"np.load"("data/reactants_stoich_coeffs.npy") # Forward molar stoichiometric coefficients
ν2 = py"np.load"("data/product_stoich_coeffs.npy") # Backward model stoichiometric coefficients
reversible = py"np.load"("data/reversible.npy")
N = size(ν1)[1]  # Number of Species
M = size(ν1)[2]  # Number of Reactions
ν1_order = zeros(N,M)
ν = ν2 - ν1  
pa = 100000 # 1 bar
R = 8314.4621 # Gas constant in kmol
### Constants: Combustion Chamber Level
min_dot = 0 # Rate at which mass enters the chamber
mout_dot = 0 # Rate at which mass leaves the chamber
Yin = ones(N) # Mass fraction of species entering the chamber
Yout = ones(N) # Mass fraction of species leaving the chamber
Qdot = 0 # Heating source
m_dot = min_dot - mout_dot
hin = 1 # Enthalpy of input
### Unknowns
T = 1200.0 # Temperature
Y = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/ini_mass_fraction.npy") # Mass fractions
### Quantities that depend on the Unknowns
tbd = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/tbd.npy") .+ 1
falofr = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/falofr.npy") .+ 1
elmr = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/elmr.npy") .+ 1

order = py"np.load"("data/reaction_orders.npy")
Af = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/pre_exponential_factor.npy") # preexponential constant Afj
β = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/temperature_exponent.npy") # Temperature exponent
E = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/activation_energy.npy") # Activation energy for the reactions in kJ

order_t = py"np.load"("data/reaction_orders_t.npy")
efficiency_t = py"np.load"("data/efficiency_t.npy")
Af_t = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/pre_exponential_factor_t.npy") # preexponential constant Afj
β_t = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/temperature_exponent_t.npy") # Temperature exponent
E_t = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/activation_energy_t.npy") # Activation energy for the reactions in kJ

order_f = py"np.load"("data/reaction_orders_f.npy")
troefall = py"np.load"("data/troefall.npy") .+ 1
troefall_coeff = py"np.load"("data/troefall_coeff.npy")
efficiency_f = py"np.load"("data/efficiency_f.npy")
Af_hi = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/pre_exponential_factor_hi.npy") # preexponential constant Afj
β_hi = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/temperature_exponent_hi.npy") # Temperature exponent
E_hi = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/activation_energy_hi.npy") 

Af_lo = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/pre_exponential_factor_lo.npy") # preexponential constant Afj
β_lo = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/temperature_exponent_lo.npy") # Temperature exponent
E_lo = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/activation_energy_lo.npy") 

ν1_order[:,elmr] = order
ν1_order[:,tbd] = order_t
ν1_order[:,falofr] = order_f;

In [238]:
function f(TY)
    T = TY[1]
    Y = TY[2:end]
    ρ = m / V # density
    X = ρ * Y ./ W # Concentration
    Q = ADCME.Variable(ones(M)) # Individual progress rates
    islarge = T > NASA_coeffs[:,1]
    islarge = convert_to_tensor(islarge, dtype=Float64)
    fhi = f_hi(NASA_coeffs, T, R)
    flo = f_lo(NASA_coeffs, T, R)
    cp = islarge .* fhi[:,1] + (1 - islarge) .* flo[:,1]
    h = islarge .* fhi[:,2] + (1 - islarge) .* flo[:,2]
    s = islarge .* fhi[:,3] + (1 - islarge) .* flo[:,3]
    cvk = cp .- R
    ΔS = ν' * s  # Entropy change for reaction j
    ΔH = ν' * h # Entahlpy change for reaction j
    ####
    M_t = efficiency_t' * X
    Kf_t = Af_t .* (T ^ β_t) .* exp(-E_t / (R * T)) .* M_t
    Kr_t = Kf_t ./ (((pa/(R * T)) ^ sum(ν[:,tbd], dims=1)')[:,1] .* exp(ΔS[tbd] ./ R - ΔH[tbd] / (R * T)))
    scatter_update(Q, tbd, Kf_t .* (prod(reshape(ADCME.repeat(X, size(tbd)[1]), (N, size(tbd)[1])) ^ order_t, dims=1)) .- Kr_t .* (prod(reshape(ADCME.repeat(X, size(tbd)[1]), (N, size(tbd)[1])) ^ ν2[:,tbd], dims=1)) .* reversible[tbd])
    ####
    M_f = efficiency_f' * X
    Kf_lo = Af_lo .* (T ^ β_lo) .* exp(-E_lo / (R * T)) .* M_f
    Kf_hi = Af_hi .* (T ^ β_hi) .* exp(-E_hi / (R * T)) 
    Pr = Kf_lo ./ Kf_hi
    Fac = ADCME.Variable(ones(size(falofr)[1]))
    for (i,s) in enumerate(troefall)
        a = troefall_coeff[:,i]
        fcent = (1 - a[1]) * exp(-T/a[2]) + a[1] * exp(-T/a[3]) + exp(-a[4]/T)
        c = -0.4 - 0.67 * log(fcent) / log(10)
        n = 0.75 - 1.27 * log(fcent) / log(10)
        f1 = (log(Pr[s]) / log(10) + c) / (n - 0.14 *(log(Pr[s]) / log(10) + c))
        scatter_update(Fac, troefall[i], 10 ^ ((log(fcent) / log(10)) / (1 + f1 ^ 2)))
    end
    Kf_f = Kf_lo ./ (1 .+ (Kf_lo ./ Kf_hi)) .* Fac
    Kr_f = Kf_f ./ (((pa/(R * T)) ^ sum(ν[:,falofr], dims=1)')[:,1] .* exp(ΔS[falofr] ./ R - ΔH[falofr] / (R * T)))
    scatter_update(Q, falofr, Kf_f .* (prod(reshape(ADCME.repeat(X, size(falofr)[1]), (N, size(falofr)[1])) ^ order_f, dims=1)) .- Kr_f .* (prod(reshape(ADCME.repeat(X, size(falofr)[1]), (N, size(falofr)[1])) ^ ν2[:,falofr], dims=1)) .* reversible[falofr][:,1])
    ####
    Kf = Af .* (T ^ β) .* exp(-E / (R * T))
    Kr = Kf ./ (((pa/(R * T)) ^ sum(ν[:,elmr], dims=1)')[:,1] .* exp(ΔS[elmr] ./ R - ΔH[elmr] / (R * T)))
    scatter_update(Q, elmr, Kf .* (prod(reshape(ADCME.repeat(X, size(elmr)[1]), (N, size(elmr)[1])) ^ order, dims=1)) .- Kr .* (prod(reshape(ADCME.repeat(X, size(elmr)[1]), (N, size(elmr)[1])) ^ ν2[:,elmr], dims=1)) .* reversible[elmr][:,1])
    ##### Computing ω_dot 
    cv = sum(cvk ./ W .* Y) # Mass heat capacities
    u = h / W - R / W * T   # Internal energy for species
    p = sum(X) * R * T # pressure
    ω_dot = W .* sum(ν .* Q', dims=2)
    ###### Species Conservation
    mgen_dot = V * ω_dot
    Y_dot = (1 / m) * ((min_dot * (Yin - Y) - mout_dot * Y) + mgen_dot) 
    ###### EnergyConservation
    T_dot = 1 / (m * cv) * (-Qdot + min_dot * (hin - sum(u .* Yin)) - p * V / m * mout_dot - sum(mgen_dot .* u))
    return T_dot, Y_dot
end

f (generic function with 1 method)

In [182]:
TY = [T;Y]

161-element Array{Float64,1}:
 1200.0
    0.006250000000000017
    0.006250000000000017
    0.006250000000000017
    0.006250000000000017
    0.006250000000000017
    0.006250000000000017
    0.006250000000000017
    0.006250000000000017
    0.006250000000000017
    0.006250000000000017
    0.006250000000000017
    0.006250000000000017
    ⋮
    0.006250000000000017
    0.006250000000000017
    0.006250000000000017
    0.006250000000000017
    0.006250000000000017
    0.006250000000000017
    0.006250000000000017
    0.006250000000000017
    0.006250000000000017
    0.006250000000000017
    0.006250000000000017
    0.006250000000000017

In [183]:
TY = Variable(TY)

PyObject <tf.Variable 'Variable_23:0' shape=(161,) dtype=float64_ref>

In [184]:
f(TY)

LoadError: .^ is disabled due to eager evaluation. Use ^ instead.

In [ ]:
T = TY[1]
Y = TY[2:end]
ρ = m / V # density
X = ρ * Y ./ W # Concentration
Q = ADCME.Variable(ones(M)) # Individual progress rates
islarge = T > NASA_coeffs[:,1]
islarge = convert_to_tensor(islarge, dtype=Float64)
fhi = f_hi(NASA_coeffs, T, R)
flo = f_lo(NASA_coeffs, T, R)
cp = islarge .* fhi[:,1] + (1 - islarge) .* flo[:,1]
h = islarge .* fhi[:,2] + (1 - islarge) .* flo[:,2]
s = islarge .* fhi[:,3] + (1 - islarge) .* flo[:,3]
cvk = cp .- R
ΔS = ν' * s  # Entropy change for reaction j
ΔH = ν' * h # Entahlpy change for reaction j
####
M_t = efficiency_t' * X
Kf_t = Af_t .* (T ^ β_t) .* exp(-E_t / (R * T)) .* M_t
Kr_t = Kf_t ./ (((pa/(R * T)) ^ sum(ν[:,tbd], dims=1)')[:,1] .* exp(ΔS[tbd] ./ R - ΔH[tbd] / (R * T)))
scatter_update(Q, tbd, Kf_t .* (prod(reshape(ADCME.repeat(X, size(tbd)[1]), (N, size(tbd)[1])) ^ order_t, dims=1)) .- Kr_t .* (prod(reshape(ADCME.repeat(X, size(tbd)[1]), (N, size(tbd)[1])) ^ ν2[:,tbd], dims=1)) .* reversible[tbd])
####
M_f = efficiency_f' * X
Kf_lo = Af_lo .* (T ^ β_lo) .* exp(-E_lo / (R * T)) .* M_f
Kf_hi = Af_hi .* (T ^ β_hi) .* exp(-E_hi / (R * T)) 
Pr = Kf_lo ./ Kf_hi
Fac = ADCME.Variable(ones(size(falofr)[1]))
for (i,s) in enumerate(troefall)
    a = troefall_coeff[:,i]
    fcent = (1 - a[1]) * exp(-T/a[2]) + a[1] * exp(-T/a[3]) + exp(-a[4]/T)
    c = -0.4 - 0.67 * log(fcent) / log(10)
    n = 0.75 - 1.27 * log(fcent) / log(10)
    f1 = (log(Pr[s]) / log(10) + c) / (n - 0.14 *(log(Pr[s]) / log(10) + c))
    scatter_update(Fac, troefall[i], 10 ^ ((log(fcent) / log(10)) / (1 + f1 ^ 2)))
end
Kf_f = Kf_lo ./ (1 .+ (Kf_lo ./ Kf_hi)) .* Fac
Kr_f = Kf_f ./ (((pa/(R * T)) ^ sum(ν[:,falofr], dims=1)')[:,1] .* exp(ΔS[falofr] ./ R - ΔH[falofr] / (R * T)))
scatter_update(Q, falofr, Kf_f .* (prod(reshape(ADCME.repeat(X, size(falofr)[1]), (N, size(falofr)[1])) ^ order_f, dims=1)) .- Kr_f .* (prod(reshape(ADCME.repeat(X, size(falofr)[1]), (N, size(falofr)[1])) ^ ν2[:,falofr], dims=1)) .* reversible[falofr][:,1])
####
Kf = Af .* (T ^ β) .* exp(-E / (R * T))
Kr = Kf ./ (((pa/(R * T)) ^ sum(ν[:,elmr], dims=1)')[:,1] .* exp(ΔS[elmr] ./ R - ΔH[elmr] / (R * T)))
scatter_update(Q, elmr, Kf .* (prod(reshape(ADCME.repeat(X, size(elmr)[1]), (N, size(elmr)[1])) ^ order, dims=1)) .- Kr .* (prod(reshape(ADCME.repeat(X, size(elmr)[1]), (N, size(elmr)[1])) ^ ν2[:,elmr], dims=1)) .* reversible[elmr][:,1])
##### Computing ω_dot 
cv = sum(cvk ./ W .* Y) # Mass heat capacities
u = h / W - R / W * T   # Internal energy for species
p = sum(X) * R * T # pressure
ω_dot = W .* sum(ν .* Q', dims=2)
###### Species Conservation
mgen_dot = V * ω_dot
Y_dot = (1 / m) * ((min_dot * (Yin - Y) - mout_dot * Y) + mgen_dot) 
###### EnergyConservation
T_dot = 1 / (m * cv) * (-Qdot + min_dot * (hin - sum(u .* Yin)) - p * V / m * mout_dot - sum(mgen_dot .* u))

In [239]:
t_dot, y_dot = f(TY)

(PyObject <tf.Tensor 'Mul_59502:0' shape=() dtype=float64>, PyObject <tf.Tensor 'Mul_59495:0' shape=(160,) dtype=float64>)

In [246]:
test = ADCME.gradients(y_dot, Y)

PyObject <tf.Tensor 'TensorArrayStack_1/TensorArrayGatherV3:0' shape=(?, 160) dtype=float64>

In [242]:
test

In [244]:
y_dot

PyObject <tf.Tensor 'Mul_59495:0' shape=(160,) dtype=float64>